# Visualització de la Informació: Projecte 1
## Comparativa de l'evolució de dues estacions de Bicing 

Autors:
* Marc Vernet
* Patricia Cabot

## Pre-procés de les dades

Hem fet servir un script a Python per fer el pre-procés. 

1. Hem descarregat les dades del mes i any que volem visualitzar: Gener de 2020.
2. Hem seleccionat les dues estacions que volem utilitzar: la 327 (situada a l'Avinguda Tibidabo, a la part alta de Barcelona, amb poca afluència) i la 31 (estació amb més afluència de Barcelona, situada a la Barceloneta). 
3. Hem substituit " . " per " _ "  als noms de les variables, ja que Altair no detecta " . ". Ho hem fet amb la funció _re.sub()_. 
4. Hem convertit la variable temporala _last_reported_ a un format temporal amb el que podem treballar, on tenim la data i la hora a cada fila. 
Ho hem realitzat amb la funció _datetime.fromtimestamp().strftime()_. 
5. Hem eliminat la variable temporal _last_updated_, ja que no la farem servir. 
6. Hem afegit una columna amb els noms dels dies de la setmana, extret de la informació de _last_reported_, que és la variable temporal utilitzada a l'hora de fer les visualitzacions. 
Ho hem realitzat amb la funció _.dt.day_name()_. 
7. Hem afegit una columna on indiquem si un dia és festiu (1) o si és laborable (0). 
8. Hem guardat les noves dades en nou fitxer anomenat _bicingGener_clean.csv_, que és l'utilitzat per les visualitzacions. 



Aquesta és la funció que s'ha utilitzat pel preprocés:

```
def dataCleaning(dataset, id1, id2, generateFile, generateDataFrame):
    #Seleccionar les estacions que ens interessen
    df1 = dataset[dataset['station_id'] == id1 ]
    df2 = dataset[dataset['station_id'] == id2 ]
    dfEstacions = df1.append(df2)
    
    #Canviar el format del titol
    dfEstacions.columns = [re.sub('\.', '_', name) for name in dfEstacions.columns]
    
    #Conversió al tipus de data Pandas (2019-12-31 23:59:06)
    dfEstacions['last_reported'] = [datetime.fromtimestamp(i).strftime("%Y-%m-%d %H:%M:%S") for i in dfEstacions['last_reported']]
    
    # Eliminem la columna 'last_updated' perquè no la utilitzem
    dfEstacions = dfEstacions.drop(columns=['last_updated'])
    
    #Afegim la columna dels dies de la setmana de cada data del last_reported
    dfEstacions['weekday'] = (pd.to_datetime(dfEstacions['last_reported'])).dt.day_name()
    
    #Indiquem quins dies corresponen al cap de setmana
    dfEstacions['weekend'] = 0
    dfEstacions.loc[(dfEstacions['weekday']=='Saturday'), 'weekend'] = 1
    dfEstacions.loc[(dfEstacions['weekday']=='Sunday'), 'weekend'] = 1
    
    #Guardar les dades processades
    if generateFile: dfEstacions.to_csv('bicingGener_clean.csv')
    if generateDataFrame:
        return dfEstacions
```

## Fase explorativa

In [ ]:
import altair as alt
import pandas as pd
from vega_datasets import data
from google.colab import drive

Primer, importem la base de dades de Bicing de Gener de 2020 prèviament netejada. 

In [ ]:
path = 'https://raw.githubusercontent.com/marcvernet31/data/main/bicingGener_clean.csv'
data = pd.read_csv(path)

In [ ]:
data.head()

,Unnamed: 0,station_id,num_bikes_available,num_bikes_available_types_mechanical,num_bikes_available_types_ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,ttl,weekday,weekend
0,28,31,2,1,1,12,1,1,1,2019-12-31 23:59:06,True,IN_SERVICE,28,Tuesday,0
1,438,31,3,2,1,11,1,1,1,2020-01-01 00:03:28,True,IN_SERVICE,11,Wednesday,0
2,848,31,1,1,0,13,1,1,1,2020-01-01 00:07:48,True,IN_SERVICE,30,Wednesday,0
3,1258,31,1,1,0,13,1,1,1,2020-01-01 00:12:07,True,IN_SERVICE,22,Wednesday,0
4,1668,31,1,1,0,13,1,1,1,2020-01-01 00:16:29,True,IN_SERVICE,7,Wednesday,0


Amb aquesta comanda, fem que Altair accepti més de 500 files. 

In [ ]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

Ara comencem a fer el procés d'anàlisi. 

In [ ]:
chart1 = alt.Chart(data).mark_line().encode(
    x = alt.Y('last_reported:T', title='Time of report'), 
    y = alt.X('num_bikes_available:Q', title='Available bikes'),
    color = alt.Color('station_id:N', legend=alt.Legend(title="Station identifier"), scale = alt.Scale(scheme='set2'))
).properties(title='All bikes based on station_id')

chart2 = alt.Chart(data).mark_line().encode(
    x = alt.Y('last_reported:T', title='Time of report'), 
    y = alt.X('num_bikes_available_types_mechanical:Q', title='Available bikes'),
    color = alt.Color('station_id:N', legend=alt.Legend(title="Station identifier"), scale = alt.Scale(scheme='set2'))
).properties(title='Mechanical bikes based on station_id')

chart3 = alt.Chart(data).mark_line().encode(
    x = alt.Y('last_reported:T', title='Time of report'), 
    y = alt.X('num_bikes_available_types_ebike:Q', title='Available bikes'),
    color = alt.Color('station_id:N', legend=alt.Legend(title="Station identifier"), scale = alt.Scale(scheme='set2'))
).properties(title='Ebikes based on station_id')

chart1 | chart2 | chart3

Quan fem un plot de totes les bicicletes disponibles sense cap mena d'agrupació, observem que la visualització és de molt baixa qualitat. Això passa perquè per cada dia tenim un gran nombre de dades diferents. 

Cal mencionar que els dies 25 i 26 no hi ha informació a la base de dades. 


Ara provem de fer els mateixos plots però, per cada dia, només indiquem la mitjana de bicis disponibles. D'aquesta manera, tenim una sola dada al dia, el que permet tenir una visió més clara de l'evolució. 

In [ ]:
chart1 = alt.Chart(data).mark_line(point=True).encode(
    x = alt.X('date(last_reported):O', title='Date'), 
    y = alt.Y('mean(num_bikes_available):Q', title='Mean available bikes'),
    color = alt.Color('station_id:N', legend=alt.Legend(title="Station identifier"), scale = alt.Scale(scheme='set2'))
). properties(title='All bikes based on station_id')

chart2 = alt.Chart(data).mark_line(point=True).encode(
    x = alt.X('date(last_reported):O', title='Date'), 
    y = alt.Y('mean(num_bikes_available_types_mechanical):Q', title='Mean available bikes'),
    color = alt.Color('station_id:N', legend=alt.Legend(title="Station identifier"), scale = alt.Scale(scheme='set2'))
).properties(title='Mechanical bikes based on station_id')

chart3 = alt.Chart(data).mark_line(point=True).encode(
    x = alt.X('date(last_reported):O', title='Date'), 
    y = alt.Y('mean(num_bikes_available_types_ebike):Q', title='Mean available bikes'),
    color = alt.Color('station_id:N', legend=alt.Legend(title="Station identifier"), scale = alt.Scale(scheme='set2'))
).properties(title='Ebike based on station_id')

chart1 | chart2 | chart3

El resultat és satisfactori. 

En general observem que la disponibilitat de bicis és molt major a l'estació 31 que a la 327. Això és degut a que la primera es troba a la Barceloneta, una zona molt concurreguda de la ciutat, mentre que l'altre es situa a la part alta de Barcelona, amb poca afluència. Té sentit, per tant, que la gent utilitzi les bicis en general per baixar i que no s'acumulin a l'estació. 

L'excepció és pel cas de les bicicletes electriques, on podem observar que la disponibilitat és major a l'estació 327. Atribuïm aquest fet a que aquesta estació es troba a la part alta de la ciutat i que en general la gent utilitza bicicletes electriques per arribar-hi, pero un cop estacionades aquestes bicicletes tenen poca demanda.

En canvi, a l'estació de la Barceloneta hi ha una alta demanda de bicicletes electriques que costa trobar, ja que la majoria de la gent arriba amb bicicleta mecànica. 


Ara seleccionem una de les estacions, en aquest cas la 31, i fem dues gràfiques, visualitzats conjuntament, per veure les diferències entre bicis disponibles mecàniques i elèctriques. 

In [ ]:
l1 = alt.Chart(data).mark_line(color = '#4DCFD1').encode(
    x = alt.X('date(last_reported):O', title='Date'), 
    y = alt.Y('mean(num_bikes_available_types_mechanical):Q', title='Mean available bikes')
).transform_filter(
    alt.FieldEqualPredicate(field = 'station_id', equal = 31)
)

l2 = alt.Chart(data).mark_line(color = '#FF9C96').encode(
    x = alt.X('date(last_reported):O', title='Date'), 
    y = alt.Y('mean(num_bikes_available_types_ebike):Q', title='Mean available bikes')
).transform_filter(
    alt.FieldEqualPredicate(field = 'station_id', equal = 31)
)

(l1 + l2).properties(title='Comparison between mechanical and electrical available bikes')

El resultat està bé, però al ser dues gràfiques diferents i haver superposat una sobre l'altre, no ens apareix la llegenda amb els colors corresponents al tipus de bicicleta. A més, tampoc podem utilitzar un únic esquema de colors. 

La solució a aquest inconvenient és crear una nova base de dades en long_form amb el tipus de bicicleta a cada fila. Això ens permet fer més gràfiques en aquest àmbit. 

In [ ]:
data_select = data[['station_id', 'last_reported', 'num_bikes_available_types_mechanical', 'num_bikes_available_types_ebike', 'weekday', 'weekend']]
data_melt = data_select.melt(('station_id', 'last_reported', 'weekday', 'weekend'), var_name='types', value_name='value')

data_melt = data_melt.replace("num_bikes_available_types_mechanical", "Mechanical")
data_melt = data_melt.replace("num_bikes_available_types_ebike", "Ebike")

data_melt['weekend'] = data_melt['weekend'].replace([0], 'Weekday')
data_melt['weekend'] = data_melt['weekend'].replace([1], 'Weekend')

data_melt.head()

KeyError: ignored

Provem de fer una gràfica de línies amb les mitjanes de cada tipus de bici. D'aquesta manera, podem veure la distrució de cada dia del mes. 

In [ ]:
alt.Chart(data_melt).mark_line(point=True).encode(
    x = alt.X('date(last_reported):O', title='Date'), 
    y = alt.Y('mean(value):Q', title='Mean available bikes'),
    color=alt.Color('types', legend=alt.Legend(title="Type of bikes"), scale = alt.Scale(range=['#4DCFD1', '#FF9C96']))
).properties(title='Available bikes based on its type')

Ara anem a observar la distribució de bicicletes segons les hores del dia. 

In [ ]:
alt.Chart(data_melt).mark_line(point=True).encode(
    x = alt.X('hours(last_reported):O', title='Hours of day'), 
    y = alt.Y('mean(value):Q', title='Mean available bikes'),
    color=alt.Color('types', legend=alt.Legend(title="Type of bikes"), scale = alt.Scale(range=['#4DCFD1', '#FF9C96']))
).properties(title="Distribution per hours of day")

Observem que l'ús de bicicletes mecàniques està molt concentrat durant el dia i a la nit disminueix dràsticament. En canvi, l'ús de bicicletes elèctriques no varia significativament, ja que el seu ús és mínim. 

Ara fem dues visualitzacions, una per cada estació, per comparar l'ús dels dos tipus de bicis per hores. A més, afegim els intervals de confiança. Ambues visualitzacions tenen la mateixa escala. 

In [ ]:
main_st_hours = alt.Chart().mark_line(point=True).encode(
    x = alt.X('hours(last_reported):T', title = 'hours'), 
    y = alt.Y('mean(value):Q', title = 'Average number of bikes'),
    color=alt.Color('types:N',  scale=alt.Scale(range=['#4DCFD1', '#FF9C96']),
        legend=alt.Legend(title="Type of bikes"))   
)

conf_st_hours = alt.Chart().mark_area(opacity=0.3).encode(
    x = 'hours(last_reported):T',
    y = 'ci0(value):Q',
    y2 = 'ci1(value):Q',
    color='types:N'  
)

alt.layer(main_st_hours + conf_st_hours, data=data_melt).facet(column='station_id')

Són unes visualitzacions molt entenedores. 
* A l'estació 31, observem com la disponibilitat de bicis mecàniques supera enormement la de bicis elèctriques, sobretot durant les hores del dia. Recordem que aquesta estació està situada a la Barceloneta i és la que te més afluència. És coherent que mostri aquests resultats. 
* A l'estació 327, trobem uns resultats bastant diferents. Primer, la disponibilitat de bicicletes comparada amb l'altre és molt menor. A més, ara les bicis mecàniques i elèctriques estan molt més igualades. De fet, durant pràcticament tot el dia, el nombre de bicis ebike és lleugerament superior. Si recordem que aquesta estació està situada a la part alta de Barcelona, aquestes dades prenen molt de sentit. És a dir, durant el dia, la gent utilitza les bicis mecàniques per baixar, per tant, no s'acumulen a l'estació. En canvi, la gent utilitza les ebike per pujar fins a la zona de l'estació i les deposita en aquesta. Per tant, sempre n'hi ha d'aquest tipus. 

Una altre cosa interessant a observar és que en els dos casos els intervals de confiança són molt estrets, es a dir que la distribució de bicicletes és bastant constant al llarg de tots els dies del mes. 

Ara provem de fer un parell de visualitzacions en funció del dia de la setmana i les hores, agregant el número de bicis disponibles. 

Primer, hem fet un bubble chart, on l'àrea de les circumferències és el nombre mitjà de bicis disponibles. A més, també diferenciem segons el tipus de bicis, superposant dues circumferències. 

In [ ]:
bubblechart = alt.Chart(data_melt).mark_circle(opacity=0.8).encode(
    x = alt.X('hours(last_reported):O', title='Hours of day'),
    y = alt.Y('weekday', type='ordinal', sort=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']),
    size = 'mean(value)',
    color = alt.Color('types:N', scale = alt.Scale(range=['#4DCFD1', '#FF9C96']), legend=alt.Legend(title="Type of bikes")),
    column = 'station_id:N'
)

bubblechart

Després, hem fet un heatmap, on la temperatura és el nombre mitjà de bicis disponibles. 

In [ ]:
heatmap_station = alt.Chart(data_melt).mark_rect().encode(
    x = alt.X('hours(last_reported):T', title='Hours of day'),
    y = alt.Y('weekday', type='ordinal', sort=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']),
    color = alt.Color('mean(value):Q', scale = alt.Scale(scheme='purples'), legend=alt.Legend(title="Average available bikes")),
    column = 'station_id:N'
)

heatmap_station.properties(width=400, height=150)

Les dues visualitzacions representen pràcticament la mateixa informació, que és el nombre mitjà de bicis disponibles en funció de la hora i del dia de la setmana. 

Tot i que ambdues representacions són bones, ens agrada més la segona, ja que els canvis entre dies i hores es veuen refletits molt millor. És a dir, tot i no poder donar valors exactes, la comparativa entre unitats temporals és més senzilla. 

Per altra banda, el bubble chart ens ha permés una altra funcionalitat, que és classificar també per tipus de bici. No obstant, en alguns casos això presenta un problema, ja que al estar el cercles superposats, hi ha moments on el cercle de davant és més gran que el de darrere i no deixa veure'l. A més, és difícil per l'ull humà estimar dues dimensions, és a dir, l'àrea de la circumeferència. 

Per tant, tot i que el heatmap no diferència entre tipus de bici, visualment és una visualització molt més encertada per fer comparacions entre dies de la setmana i hores del dia. 



Observem que la disponibilitat de bicis a l'estació 31 és major. Durant el dia observem un augment considerable. No s'observen grans diferències entre els dies laborables i els caps de setmana. Es pot esmentar que a la mitja nit de dijous a divendres hi ha més presència de bicis i el cap de setmana la presència de bicis augmenta a la matinada desde les 2h del matí, mentre que els dies laborables ho fa a les 6h.

En canvi, veiem que a l'altre estació no es produeix una gran variació durant el dia ni entre els dies de la setmana. Només es pot destacar un breu augment del nombre de bicis a les hores de la matinada i al cap de setmana, que sembla que hi ha més presència. Aquest fet deu ser perquè la gent no ha d'anar a treballar o a l'escola i simplement el nombre de bicis es queda més o menys igual. 

Segurament els augments puntuals que es poden veure son causats pel servei de manteniment del Bicing, que transporta bicicletes cap a estacions amb falta de bicicletes.


Ara anem fer visualitzacions on classifiquem conjuntament per tipus de bici i per identificador de l'estació. 

Primer provem amb una visualització on apilem dades dels dos tipus de bicis. 

In [ ]:
a = alt.Chart(data_melt[data_melt['station_id'] == 31]).mark_area(opacity=0.85).encode(
    x = alt.X('date(last_reported):T', title='Date'), 
    y = alt.Y('mean(value):Q', title='Mean available bikes'),
    color=alt.Color('types:N',  scale=alt.Scale(range=['MediumAquaMarine', 'Steelblue']),
        legend=alt.Legend(title="Station 31"))   
)

b = alt.Chart(data_melt[data_melt['station_id'] == 327]).mark_area(opacity=0.85).encode(
    x = alt.X('date(last_reported):T', title='Date'), 
    y = alt.Y('mean(value):Q', title='Mean available bikes'),
    color=alt.Color('types:N',  scale=alt.Scale(range=['LightSalmon', 'Crimson']),
        legend=alt.Legend(title="Station 327"))
)

(a + b).resolve_scale(color="independent").properties(title='Available bikes based on station id and type', width=550, height=300)

Visualment és atractiva, però observem dos problemes:
1. Al ser dades apilades, el que està a sobre no es pot comparar bé amb la resta. 
2. A l'estació 31, el nombre de bicis mecàniques disponibles és molt alt sempre i, per tant, resalta sobre la resta. Pot succeir que ens fixem només en aquestes dades i no en els altres. 

Ara provem de canviar les dades apilades per dades en línies, ja que creiem que visualment serà més agradable. 

In [ ]:
a = alt.Chart(data_melt).mark_line(point=True).encode(
    x = alt.X('date(last_reported):O', title='Date'), 
    y = alt.Y('mean(value):Q', title='Mean available bikes'),
    color=alt.Color('types',  scale=alt.Scale(range=['#4DCFD1', '#3C9ABD']),
        legend=alt.Legend(title="Station 31"))
).transform_filter(
    alt.FieldEqualPredicate(field = 'station_id', equal = 31)
)

b = alt.Chart(data_melt).mark_line(point=True).encode(
    x = alt.X('date(last_reported):O', title='Date'), 
    y = alt.Y('mean(value):Q', title='Mean available bikes'),
    color=alt.Color('types',  scale=alt.Scale(range=['#FF9C96', '#EB435D']),
        legend=alt.Legend(title="Station 327"))
).transform_filter(
    alt.FieldEqualPredicate(field = 'station_id', equal = 327)
)

(a+b).resolve_scale(color="independent").properties(title='Available bikes based on station id and type')

Es veu molt millor l'evolució de les bicis disponibles amb aquesta representació. No obstant, al ser dues gràfiques diferents, hem hagut d'arreglar l'escala de colors perquè siguin independents i poguem distingir entre les dues estacions. 

És una visualització molt entenedora del que ha passat al llarg del mes en relació al nombre de bicis disponibles en funció de l'estació i del tipus. En general s'observa que l'ús de bicis mecàniques a l'estació 31 supera significativament el de la resta. També podem apreciar que la disponibilitat de bicis mecàniques a l'estació 327 recull pics i baixades molt grans. En canvi, l'ús de les bicis ebike és bastant constant, no presenta grans variacions, el qual és major a l'estació 327 com ja havíem observat en anteriors visualitzacions. 

A continuació, intentem solucionar el problema de l'escala de colors realitzant la mateixa visualització en una sola gràfica. 

A més, repetim la visualització per dues mesures de temps més a part de la data: els dies de la setmana i les hores del dia. 

In [ ]:
month = alt.Chart(data_melt).transform_calculate(
    station_type = alt.datum.station_id +' - ' + alt.datum.types
).mark_line(point=True).encode(
    x = alt.X('date(last_reported):O', axis=alt.Axis(title="Day of month")),
    y = alt.Y('mean(value):Q', axis=alt.Axis(title="Number of available bikes")),
    color = alt.Color('station_type:N', scale = alt.Scale(range=['#4DCFD1', '#3C9ABD', '#FF9C96', '#EB435D']), 
                    legend=alt.Legend(title="Station with types of bikes"))
)

week = alt.Chart(data_melt).transform_calculate(
    station_type = alt.datum.station_id +' - ' + alt.datum.types,
    dia = "day(datum.last_reported)"
).mark_line(point=True).encode(
    x = alt.X('weekday', type='ordinal', sort=['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'], axis=alt.Axis(title="Day of week")),
    y = alt.Y('mean(value):Q', axis=alt.Axis(title="Number of available bikes")),
    color = alt.Color('station_type:N', scale = alt.Scale(range=['#4DCFD1', '#3C9ABD', '#FF9C96', '#EB435D']), legend=alt.Legend(title="Station with types of bikes")),
    order = 'dia:O'
).properties(title="Number of available bikes during the week", width=500)

hours = alt.Chart(data_melt).transform_calculate(
    station_type = alt.datum.station_id +' - ' + alt.datum.types
).mark_line(point=True).encode(
    x = alt.X('hours(last_reported):O', axis=alt.Axis(title="Hours of day")),
    y = alt.Y('mean(value):Q', axis=alt.Axis(title="Number of available bikes")),
    color = alt.Color('station_type:N', scale = alt.Scale(range=['#4DCFD1', '#3C9ABD', '#FF9C96', '#EB435D']), 
                    legend=alt.Legend(title="Station with types of bikes"))
).properties(title="Number of available bikes during the day")

month | week | hours

Com ara tenim tota la informació dins de cada gràfica, hem pogut posar el mateix rang de colors a totes les variables. 

És agradable visualment aquest tipus de representació, alhora que és molt senzill fer comparacions entre les diferents classificacions de les dades. 

A grosso modo, podem afirmar que la disponibilitat de bicis mecàniques a l'estació 31 supera de manera important la de la resta. 

Ara provem de fer la gràfica mensual amb unes marques que indiquen quan comença cada setmana. 

In [ ]:
startweek = alt.Chart(data_melt).mark_rule(opacity=0.3, color='grey').encode(
    x = 'date(last_reported):O'
).transform_filter(
    alt.FieldEqualPredicate(field='weekday', equal='Monday')
)

month+startweek

També fem unes visualitzacions per comparar els diferents dies del mes entre ells. És a dir, tots els dilluns, tots els dimarts... Per tal de veure si es comporten igual. 

In [ ]:
days = alt.Chart(data_melt).mark_line(point=True).transform_calculate(
    station_type = alt.datum.station_id +' - ' + alt.datum.types
).encode(
    x = alt.X('date(last_reported):T', title='Date'),
    y = alt.Y('mean(value):Q', title='Average number of bikes'),
    color = alt.Color('station_type:N',  scale = alt.Scale(range=['#4DCFD1', '#3C9ABD', '#FF9C96', '#EB435D']), legend = alt.Legend(title="Station with types of bikes")),
    column = alt.Column('weekday:O', title="Day evolution comparison", sort=alt.SortField("diaord:O", order="ascending"))
).properties(width=100, height=100)

days

Ambdues visualitzacions mostre l'evolució mensual dels dies. Considerem que per realitzar la tasca de comparar els dies de les setmanes que conformen el mes és millor la segona representació, ja que les comparacions són més senzilles de realitzar.

Observem com els dies entre ells en general segueixen una continuïtat. És a dir, tenen un comportament que podem considerar constant. Tot i així, trobem algunes excepcions. 
* El dimarts observem que les bicis mecàniques de l'estació 31 tenen un comportament constant però, a l'últim del mes, la seva disponibilitat disminueix de manera significativa. 
* El quart dimecres de mes notem un lleuger pic de bicis mecàniques de l'estació 327. 
* El primer divendres del mes, les bicis mecàniques de l'estació 327 tenen un nombre molt alt i després disminueix considerablement, per estar quasi a zero. 
* El dissabte veiem que les bicis mecàniques de l'estació 327 al principi de mes tenen una disponibilitat reduïda però el tercer dissabte aquesta creix considerablement. 
* El dilluns observem com la presència de bicis mecàniques a l'estació 31 augmenta a mesura que avança el mes. 

El que sí que cal destacar és que tots els dies mantenen una disponibilitat del tipus de bici i de l'estació en un mateix rang de magnituds. Per tant, no hi ha grans variacions de valors entre els diferents dies de la setmana ni entre els mateixos dies de diferents setmanes. 

Ara provem de fer una visualització classificant per dies laborables o festius. 

In [ ]:
main_week = alt.Chart().mark_line(point=True).transform_calculate(
    station_type = alt.datum.station_id +' - ' + alt.datum.types
).encode(
    x = alt.X('hours(last_reported):O', title = 'Hours'), 
    y = alt.Y('mean(value):Q', title = 'Average available bikes'),
    color=alt.Color('station_type:N',  scale=alt.Scale(range=['#4DCFD1', '#3C9ABD', '#FF9C96', '#EB435D']),
        legend=alt.Legend(title="Station with types of bikes"))   
).properties(width = 400, height=200)

conf_week = alt.Chart().mark_area(opacity=0.2).transform_calculate(
    station_type = alt.datum.station_id +' - ' + alt.datum.types
).encode(
    x = 'hours(last_reported):O',
    y = 'ci0(value):Q',
    y2 = 'ci1(value):Q',
    color=alt.Color('station_type:N',
        legend=alt.Legend(title="Station with types of bikes"))   
).properties(width = 400, height=200)

alt.layer(main_week + conf_week, data=data_melt).facet(column='weekend')

Aquesta visualització és una comparativa molt completa, ja que recull l'evolució de la disponibilitat de bicis classificada per estació i tipus de bici segons les hores del dia i, a més, separada en dues gràfiques per comparar entre dies laborables i festius. 

A grosso modo, podem observar un comportament semblant entre les dues gràfiques. No obstant, es poden destacar alguns fets. 
* Pel que fa a les bicis mecàniques en ambdues estacions, l'agument en la presència de bicis als dies festius es dona hores abans comparat als dies laborables. També cal esmentar que la variabilitat (indicat a l'interval de confiança) del nombre de bicis incrementa als caps de setmana. 
A més, en el cas concret de l'estació 327, la disponibilitat de bicis als matins de dies festius es manté constant fins al migdia. Això no passa als dies d'entre setmana, que la disponibilitat comença a caure a partir de les 7. És degut a que són dies laborables i la gent agafa la bicicleta per anar a treballar o a l'escola. 
* Per altra banda, el nombre de bicis ebike a l'estació 327 té un increment notable als matins dels caps de setmana fins al migdia. En canvi, les variacions als dies laborables és mínima.
* Finalment, la disponibilitat de bicis ebike a l'estació 31 es manté constant durant tota la setmana, independentment de si és o no laborable, sempre en nombres molt baixos. 

Ara provem de fer la mateixa visualització però separant en dues gràfiques verticals cada estació. 

In [ ]:
week_st = alt.Chart().mark_line(point=True).encode(
    x = alt.X('hours(last_reported):O', title = 'Hours'), 
    y = alt.Y('mean(value):Q', title = 'Average number of bikes'),
    color=alt.Color('types:N',  scale=alt.Scale(range=['#4DCFD1', '#FF9C96']), legend=alt.Legend(title="Station with types of bikes"))   
).properties(width = 400, height=150)

conf_st = alt.Chart().mark_area(opacity=0.2).encode(
    x = 'hours(last_reported):O',
    y = 'ci0(value):Q',
    y2 = 'ci1(value):Q',
    color=alt.Color('types:N',  scale=alt.Scale(range=['#4DCFD1', '#FF9C96']), legend=alt.Legend(title="Station with types of bikes"))   
)

st_31 = alt.layer(week_st+conf_st, data=data_melt).facet(column='weekend').transform_filter(alt.FieldEqualPredicate(field='station_id', equal=31)).properties(title='Estació 31')
st_327 = alt.layer(week_st+conf_st, data=data_melt).facet(column='weekend').transform_filter(alt.FieldEqualPredicate(field='station_id', equal=327)).properties(title='Estació 327')

alt.vconcat(st_31, st_327).resolve_scale(y='shared')

Ambues visualitzacions són bones. Tot i que a la segona es veu menys atapeït perquè hem separat per estacions, alhora de fer comparacions és millor tenir la informació compacta i, per això, creiem que la primera visualització és més adient per la tasca que hem de realitzar. A més, a la segona visualització, al haver de tenir la mateixa escala de valors per poder comparar valors, a la segona fila (estació 327) es veuen les dades molt comprimides en un espai petit i la majoria de l'espai a les gràfiques està en blanc. Per tant, la primera visualització és millor. 

Ara provem de fer visualitzacions completes de tota la setmana per obtenir un "overview" de l'ús de bicis. 

In [ ]:
week_day = alt.Chart(data_melt).mark_line().transform_calculate(
    station_type = alt.datum.station_id +' - ' + alt.datum.types
).transform_timeunit(
    datetime_day = "day(last_reported)"
).transform_calculate(
    day_number = "day(datum.last_reported)", 
    day_name = "split(datum.datetime_day,' ')[0]",
    day_hour = "(datum.day_name) + ' ' + hours(datum.last_reported) + ':00:00'"
).encode(
    x = alt.X('day_hour:O', sort=alt.SortField(field="day_number"), title='Day and hour'),
    y = alt.Y('mean(value):Q', title='Average available bikes'),
    order = "day_number:Q",
    color = alt.Color('station_type:N',  scale = alt.Scale(range=['#4DCFD1', '#3C9ABD', '#FF9C96', '#EB435D']), legend = alt.Legend(title="Station with types of bikes"))
)

confi_weekday = alt.Chart(data_melt).mark_area(opacity=0.2).transform_calculate(
    station_type = alt.datum.station_id +' - ' + alt.datum.types
).transform_timeunit(
    datetime_day = "day(last_reported)"
).transform_calculate(
    day_number = "day(datum.last_reported)", 
    day_name = "split(datum.datetime_day,' ')[0]",
    day_hour = "(datum.day_name) + ' ' + hours(datum.last_reported) + ':00:00'"
).encode(
    x = alt.X('day_hour:O', sort=alt.SortField(field="day_number"), title='Day and hour'),
    y = 'ci0(value):Q',
    y2 = 'ci1(value):Q',
    order = "day_number:Q",
    color = 'station_type:N'
)

(week_day+confi_weekday).properties(title='Weekly bike usage', width = 800, height=200)

En aquesta visualització podem observar l'evolució de la disponibilitat de bicis al llarg de la setmana i, a més, hem afegit les hores de cada dia. És a dir, mostra un resum molt complet de les dades. Per aquesta raó hem deciti que aquesta sigui la visualització que mostrem com a overview. 

Pel que fa a les bicis de l'estació 31 observem quelcom evident: que existeix un patró estacional, on hi ha un creixement a les hores del dia i un descens significatiu a la nit. El més important és que aquest patró es manté per a tots els dies, cosa que no succeeix amb la resta de bicis. 

A diferència de les anteriors, les bicis mecàniques de l'estació 327 no tenen un patró diari, ja que d'alguna manera sembla que el mostrin la majoria de dies però en canvi dilluns i dimarts no. Per tant, és un comportament bastant diferent. A més, la seva disponibilitat és molt inferior al de les bicis del mateix tipus de l'altre estació. Podem esmentar que, excepte dilluns i dimarts, la resta de dies mostren un increment de bicis sobre les 4 de la matinada i un decreixement a primera hora del matí, sobre les 8.30. Es deu a que, com ja s'ha mencionat en altres apartats, la gent utilitza aquestes bicis per anar al treball o l'escola, ja que l'estació se sitúa a la part alta de la ciutat i aquestes bicis van bé per baixar. 

Finalment, les bicis elèctriques, independentment de l'estació, tenen una presència molt inferior respecte les mecàniques. Tenen augments o descens, però són insignificants comparats amb les magnituds de l'altre tipus de bici. Només en algunes hores del dia s'observa més bicis ebike que mecàniques a l'estació 327 i és quan aquestes segones estan en devallada i la seva presència és quasi nul·la. 

En general, la disponibilitat de bicis ebike és molt inferior respecte el de bicis mecàniques. En concret, les bicis mecàniques de l'estació 31 superen significativament la presència de la resta de bicis. 

In [ ]:
 week_day2 = alt.Chart(data_melt).mark_line().transform_timeunit(
    datetime_day = "day(last_reported)"
).transform_calculate(
    day_number = "day(datum.last_reported)", 
    day_name = "split(datum.datetime_day,' ')[0]",
    day_hour = "(datum.day_name) + ' ' + hours(datum.last_reported) + ':00:00'"
).encode(
    x = alt.X('day_hour:O', sort=alt.SortField(field="day_number"), title='Day and hour'),
    y = alt.Y('mean(value):Q', title='Average number of bikes'),
    order = "day_number:Q",
    color = alt.Color('types:N',  scale = alt.Scale(range=['#4DCFD1', '#FF9C96']), legend = alt.Legend(title="Station with types of bikes")),
    row = 'station_id'
)

week_day2.properties(width = 800, height=150)

Ambdues visualitzacions són molt completes. Podem observer l'evolució de les bicis disponibles al llarg de la setmana i de les hores del dia. No obstant, creiem que per tal d'obtenir un overview de les dades és millor obtenir la informació d'una sola gràfica, ja que d'aquesta manera no cal anar movent la vista abaix i a dalt. A més, també és important mencionar que a la segona visualització perdem molt espai en blanc a la gràfica d'abaix, ja que ambdues han de tenir la mateixa escala. 
D'aquesta manera, la primera visualització és més adequada per la tasca.

## Infografia
A continuació recollim, en una infografia, el conjunt de visualitzacions que creiem que representen les dades correctament. 


<img src='https://drive.google.com/uc?id=1aPRLl1Ii6BZ-dn1N1HMVmpb0qN-om3g5' width="800">








